# Importing Dependencies

In [1]:
import os
import numpy as np

import cv2
import matplotlib.pylab as plt

import tensorflow as tf

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

# Importing Evaluation Metrices 

In [3]:
from tensorflow.keras.metrics import Precision, Recall, SpecificityAtSensitivity

pre = Precision()
rec = Recall()
spe = SpecificityAtSensitivity(0.5)

# Creating CNN Model

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.regularizers import l2

In [15]:
model = Sequential()

model.add(Conv2D(16, (3,3), 1, activation='relu', input_shape=(512,512,3)))
model.add(MaxPooling2D())
model.add(Dropout(0.4))

# model.add(Conv2D(32, (3,3), 1, activation='relu'))
# model.add(MaxPooling2D())
# model.add(Dropout(0.5))

model.add(Conv2D(8, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())
model.add(Dropout(0.3))

model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))

model.add(Dense(1, kernel_regularizer=l2(0.001), activation='linear'))

model.compile(optimizer='adam', loss='hinge', metrics='accuracy')

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 510, 510, 16)      448       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 255, 255, 16)     0         
 2D)                                                             
                                                                 
 dropout_6 (Dropout)         (None, 255, 255, 16)      0         
                                                                 
 conv2d_5 (Conv2D)           (None, 253, 253, 8)       1160      
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 126, 126, 8)      0         
 2D)                                                             
                                                                 
 dropout_7 (Dropout)         (None, 126, 126, 8)      

In [6]:
logdir = 'LogDir'

try:
    os.mkdir(logdir)
except Exception as e:
    pass    

tfb_callback = tf.keras.callbacks.TensorBoard(logdir)

# Loading Dataset

In [7]:
train_dir = 'imgdata_v2/split_dataset/train_aug/'
val_dir = 'imgdata_v2/split_dataset/val/'
test_dir = 'imgdata_v2/split_dataset/test/'

In [16]:
train_dataset = tf.keras.utils.image_dataset_from_directory(train_dir, image_size=(512, 512))
val_dataset = tf.keras.utils.image_dataset_from_directory(val_dir, image_size=(512, 512))

Found 7051 files belonging to 2 classes.
Found 415 files belonging to 2 classes.


In [17]:
print(train_dataset.as_numpy_iterator().next()[0][0].shape)
print(train_dataset.as_numpy_iterator().next()[0][0].max())
print(train_dataset.as_numpy_iterator().next()[0][0].min())

(512, 512, 3)
255.0
0.0


In [18]:
train_data = train_dataset.map(lambda x, y: (x/255, y))
val_data = val_dataset.map(lambda x, y: (x/255, y))

In [19]:
print(train_data.as_numpy_iterator().next()[0][0].shape)
print(train_data.as_numpy_iterator().next()[0][0].max())
print(train_data.as_numpy_iterator().next()[0][0].min())

(512, 512, 3)
1.0
0.0


# Training Non-Augmented Model

In [ ]:
model_fit = model.fit(train_data, 
                      epochs=30,
                      # batch_size=16,
                      shuffle=True, 
                      validation_data=val_data, 
                      callbacks=[tfb_callback])

Epoch 1/30
221/221 [==============================] - 22s 94ms/step - loss: 1.7625 - accuracy: 0.7657 - val_loss: 0.7592 - val_accuracy: 0.8096
Epoch 2/30
221/221 [==============================] - 21s 92ms/step - loss: 0.3554 - accuracy: 0.8446 - val_loss: 0.7385 - val_accuracy: 0.8193
Epoch 3/30
193/221 [=========================>....] - ETA: 2s - loss: 0.2916 - accuracy: 0.8706

# Plotting Model Performance

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(20,5))

axs[0].plot(model_fit.history['loss'], 
            color='red', 
            label='Loss')
axs[0].plot(model_fit.history['val_loss'], 
            color='orange', 
            label='Validation Loss')
axs[0].legend(loc='upper right')

axs[1].plot(model_fit.history['accuracy'], 
             color='green', 
             label='Accuracy')
axs[1].plot(model_fit.history['val_accuracy'], 
             color='teal', 
             label='Validation Accuracy')
axs[1].legend(loc='upper left')

# Evaluating Model Performance

In [ ]:
test_dataset = tf.keras.utils.image_dataset_from_directory(test_dir, image_size=(256, 256))

In [ ]:
test_data = test_dataset.map(lambda x, y: (x/255, y))

In [ ]:
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy, SpecificityAtSensitivity

pre = Precision()
acc = BinaryAccuracy()
rec = Recall()
spe = SpecificityAtSensitivity(0.5)

In [ ]:
for batch in test_data.as_numpy_iterator():
    X, y = batch
    y_pred = model.predict(X)
    pre.update_state(y, y_pred)
    rec.update_state(y, y_pred)
    acc.update_state(y, y_pred)
    spe.update_state(y, y_pred)

In [ ]:
precision = pre.result().numpy()
recall = rec.result().numpy()
f1 = 2 * (precision*recall) / (precision+recall)

print('Accuracy\t:', acc.result().numpy())
print('Precision\t:', precision)
print('Recall\t\t:', recall)
print('Specificity\t:', spe.result().numpy())
print('F1\t\t:', f1)

# ----------------------------------